# Random Forest Forecast (M4 Data)

This notebook evaluates a **Random Forest** model on a subset of the M4 Financial dataset. 

### Methodology
- **Local Training**: A separate RF model is trained for each time series (matching the NN-kNN and ARIMA baselines).
- **Feature Engineering**: Unlike the raw-window approach of NN-kNN, we use technical indicators (RSI, MA, MACD, Volatility) generated by the `FeatureBuilder`.
- **Multi-Output Forecasting**: The model is trained to predict the next $H$ steps simultaneously (Direct/Multi-output strategy).
- **Interpretability**: We extract explicit decision rules and feature importance to demonstrate the "white-box" nature of the tree model.

In [51]:
import pandas as pd
import numpy as np
from joblib import Parallel, delayed
from sklearn.preprocessing import StandardScaler
import sys
from pathlib import Path

current_dir = Path.cwd()
src_path = current_dir.parent / 'src'
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

from models.rf_model import RFTrainer
from data.tech_indicators import FeatureBuilder
from evaluation.rf_interpretability import RFExplainer
from utils import clean_daily_series, rmse, mae, smape, mase, directional_accuracy, print_evaluation_table

In [52]:
daily_train = pd.read_csv('../src/data/m4_data/Daily-train.csv')
daily_test = pd.read_csv('../src/data/m4_data/Daily-test.csv')

In [53]:
def prepare_rf_data(ts, H=14):
    """
    Prepares features and multi-step targets for Random Forest.
    
    Args:
        ts (np.array): Univariate time series.
        H (int): Forecast horizon (number of steps to predict).
        
    Returns:
        X (np.array): Feature matrix [N, D]
        y (np.array): Target matrix [N, H] (Multi-output)
        feat_names (list): List of feature names
        scaler (StandardScaler): Fitted scaler for features
    """
    fb = FeatureBuilder(
        ma_periods=[7, 30], 
        rsi_p=14, 
        vol_p=20, 
        lags=[1, 7]
    )

    features, names = fb.build(ts)
    
    # Need to align X[t] with y[t] = [ts[t+1], ..., ts[t+H]]
    # the FeatureBuilder consumes the first 'min_len' points, so features correspond to ts[end-len(features):]
    valid_ts_start = len(ts) - len(features)
    ts_aligned = ts[valid_ts_start:]
    
    X, y = [], []
    
    # Sliding window to create targets
    # We stop when we can't form a full target vector of size H
    for i in range(len(features) - H):
        X.append(features[i])
        # Target is the next H values
        y.append(ts_aligned[i+1 : i+1+H])
        
    X = np.array(X)
    y = np.array(y)
    
    # 3. Scaling (Critical for convergence/consistency, though less strictly required for Trees)
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    return X_scaled, y, names, scaler, fb

In [54]:
def forecast_rf(train_ts, H=14):
    """
    Trains a local RF model and forecasts H steps ahead.
    """
    # 1. Prepare Training Data
    try:
        X_train, y_train, feat_names, scaler, fb = prepare_rf_data(train_ts, H=H)
    except ValueError:
        # Handle cases where series is too short for FeatureBuilder
        return np.zeros(H)
        
    # 2. Train Random Forest (Multi-output)
    # Using specific hyperparameters for stability on small financial datasets
    trainer = RFTrainer(
        n_trees=100, 
        max_depth=10, 
        min_samples_leaf=3, # slight regularization
        random_state=42
    )
    
    trainer.fit(X_train, y_train, feat_names=feat_names, verbose=False)
    
    # 3. Predict
    # We need the *latest* feature vector to predict the future
    # We reconstruct features for the FULL series to get the very last point
    full_feats, _ = fb.build(train_ts)
    last_feat = full_feats[-1].reshape(1, -1)
    last_feat_scaled = scaler.transform(last_feat)
    
    forecast = trainer.predict(last_feat_scaled)
    
    return forecast.flatten(), trainer

In [55]:
def evaluate_forecast(train_row, test_row, H=14):
    train_ts = clean_daily_series(train_row).to_numpy()
    test_ts = clean_daily_series(test_row).to_numpy()

    # Enforce M4 horizon limit
    H_eval = min(H, len(test_ts))
    
    # Forecast
    y_pred, _ = forecast_rf(train_ts, H=H_eval)
    
    # Handle edge case where model failed (zeros returned)
    if np.all(y_pred == 0):
        return (np.nan, np.nan, np.nan, np.nan, np.nan)

    # Metrics
    y_true = test_ts[:H_eval]
    y_pred = y_pred[:H_eval]

    return (
        rmse(y_true, y_pred),
        mae(y_true, y_pred),
        smape(y_true, y_pred),
        mase(y_true, y_pred, train_ts, m=7),
        directional_accuracy(y_true, y_pred)
    )

In [56]:
horizons = [1, 7, 14]
NUM_SERIES = 50

daily_train_copy = daily_train.copy()
daily_test_copy = daily_test.copy()

all_results = {}

print(f"Starting Random Forest Evaluation on {NUM_SERIES} series...")

for H in horizons:
    print(f"\nEvaluating horizon H={H}")
    results = Parallel(n_jobs=-1, backend="loky", verbose=5)(
        delayed(evaluate_forecast)(daily_train_copy.iloc[i], daily_test_copy.iloc[i], H=H)
        for i in range(NUM_SERIES)
    )
    all_results[H] = results

Starting Random Forest Evaluation on 50 series...

Evaluating horizon H=1


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
/Users/johannguyen/Classes/Fall_25/559/Project/nn-knn-tree-forecasting/.venv/lib/python3.14/site-packages/sklearn/base.py:1336: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Users/johannguyen/Classes/Fall_25/559/Project/nn-knn-tree-forecasting/.venv/lib/python3.14/site-packages/sklearn/base.py:1336: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Users/johannguyen/Classes/Fall_25/559/Project/nn-knn-tree-forecasting/.venv/lib/python3.14/site-packages/sklearn/base.py:1336: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for e


Evaluating horizon H=7


[Parallel(n_jobs=-1)]: Done  38 out of  50 | elapsed:    1.9s remaining:    0.6s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    2.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.



Evaluating horizon H=14


[Parallel(n_jobs=-1)]: Done  38 out of  50 | elapsed:    2.0s remaining:    0.6s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    2.2s finished


In [57]:
for H, results in all_results.items():
    valid_results = [r for r in results if not np.isnan(r[0])]
    if not valid_results:
        print(f"\nHorizon: {H} - No valid results.")
        continue 
    rmses, maes, smapes, mases, das = zip(*valid_results)

    print(f"\nHorizon: {H}")
    print_evaluation_table(rmses, maes, smapes, mases, das)


Horizon: 1
      Metric     Mean   Median
0       RMSE  72.4041  25.3520
1        MAE  72.4041  25.3520
2  sMAPE (%)   2.1223   0.7717
3       MASE   0.6592   0.3360
4         DA      NaN      NaN

Horizon: 7
      Metric      Mean   Median
0       RMSE  198.9710  74.8692
1        MAE  185.3174  68.1317
2  sMAPE (%)    4.7159   1.8990
3       MASE    1.5385   0.8760
4         DA    0.5900   0.5000

Horizon: 14
      Metric      Mean    Median
0       RMSE  246.2043  118.4090
1        MAE  220.0085  100.6836
2  sMAPE (%)    6.0331    2.8555
3       MASE    2.2993    1.2814
4         DA    0.5123    0.5000


/Users/johannguyen/Classes/Fall_25/559/Project/nn-knn-tree-forecasting/experiments/utils.py:93: RuntimeWarning: Mean of empty slice
  np.nanmean(das)
/Users/johannguyen/Classes/Fall_25/559/Project/nn-knn-tree-forecasting/experiments/utils.py:100: RuntimeWarning: All-NaN slice encountered
  np.nanmedian(das)


## Interpretability Showcase
Here we select one series and demonstrate the explainability of the Random Forest model by extracting:
1. **Feature Importance**: Which technical indicators mattered most?
2. **Decision Rules**: Explicit IF-THEN logic used for the forecast.

In [58]:
# === Interpretability Showcase ===
# We use Horizon=1 here because the Explainability tool is designed for 
# single-target analysis (explaining "Next Day Return" rules).

# 1. Pick a sample series and prepare SINGLE-STEP data (H=1)
sample_idx = 0
train_sample = clean_daily_series(daily_train.iloc[sample_idx]).to_numpy()
H_sample = 1 

print(f"Analyzing Series {daily_train.iloc[sample_idx]['V1']} (Horizon={H_sample})...")

# 2. Regenerate data for H=1
# We flatten y to shape (N,) so the model acts as a standard single-target regressor
X_sample, y_sample, feat_names, scaler, fb = prepare_rf_data(train_sample, H=H_sample)
y_sample = y_sample.ravel() 

# 3. Retrain a local model explicitly for this demo
trainer = RFTrainer(n_trees=100, max_depth=10, min_samples_leaf=3, random_state=42)
trainer.fit(X_sample, y_sample, feat_names=feat_names, verbose=False)

# 4. Initialize Explainer
explainer = RFExplainer(trainer)
# Now predictions will be 1D, matching the internal logic of the explainer
report = explainer.generate_report(X_test=X_sample, y_test=y_sample) 

# 5. Feature Importance
print("\n=== Top 5 Feature Importance ===")
for item in report['importance']['ranking'][:5]:
    print(f"{item['rank']}. {item['feature']:<12} Score: {item['importance']:.4f}")

# 6. Extract Rules
print("\n=== Sample Decision Rules ===")
# FIX: Access .model directly to pass 'min_samples'. 
# We lower min_samples to 2 because we are training on small local datasets.
rules = trainer.model.extract_rules(max_rules=3, min_samples=2)

if not rules:
    print("No rules found. (Try increasing max_depth or lowering min_samples_leaf)")
else:
    for i, r in enumerate(rules):
        # Clean up the string for display
        rule_str = r['rule_str']
        if len(rule_str) > 100: rule_str = rule_str[:100] + "..."
        
        # Show rule with its weight (prediction value)
        print(f"Rule {i+1}: {rule_str}")
        print(f"        [Based on {r['n_samples']} samples, Prediction: {r['prediction']:.4f}]")
    
# 7. Contrastive Rules (What separates HIGH vs LOW predictions?)
print("\n=== Contrastive Rules (High vs Low Forecasts) ===")
# These rules explain what drives the model to predict distinctively high or low returns
for i, rule in enumerate(report['contrastive_rules'][:3]):
    print(f"  {i+1}. [{rule['direction']}] disc={rule['discrimination']:.3f} | {rule['rule']}")

Analyzing Series D1 (Horizon=1)...

=== Top 5 Feature Importance ===
1. ma_30        Score: 0.4082
2. ma_7         Score: 0.2660
3. lag_1        Score: 0.2630
4. lag_7        Score: 0.0619
5. vol          Score: 0.0004

=== Sample Decision Rules ===
Rule 1: IF lag_7 > 0.125 AND lag_1 > 0.985 AND lag_7 > 1.386 AND lag_1 > 1.789 AND lag_7 > 1.798 AND macd <=...
        [Based on 3 samples, Prediction: 2027.8000]
Rule 2: IF ma_7 > 0.108 AND ma_7 > 0.896 AND ma_30 > 1.411 AND lag_1 > 1.751 AND vol <= -0.170 AND lag_1 > 1...
        [Based on 3 samples, Prediction: 2025.6333]
Rule 3: IF ma_30 > 0.099 AND lag_1 > 0.924 AND ma_7 > 1.431 AND lag_1 > 1.756 AND vol <= -0.145 AND ret_5 <=...
        [Based on 4 samples, Prediction: 2024.5000]

=== Contrastive Rules (High vs Low Forecasts) ===
  1. [LOW] disc=0.718 | IF ma_7 <= -0.093 AND lag_1 <= -0.864 AND ma_7 > -1.221 AND ma_7 > -1.082 AND macd_sig > -1.204 AND macd > -0.948 AND ma_30 > -1.102 AND macd_sig > -1.023 AND lag_1 <= -1.002 AND rsi 